# Stacks and Queues

## Agenda

1. Stacks
     - ... for delimiter pairing
     - ... for postfix expression evaluation
     - ... for tracking execution and *backtracking*
2. Queues
    - ... for tracking execution and *backtracking*
    - ... for apportioning work
    - ... for fair scheduling (aka "round-robin" scheduling)
3. Run-time analysis
4. Closing remarks

## Overview

While the list data structure is incredibly useful, both implementations we explored (array-backed and linked) have operations that run in $O(N)$ time, which make them non-ideal for use with large, growing collections of data.

By further restricting the list API, however — in particular, by *isolating points of access to either the front or end of the data set* — we can create data structures whose operations are all $O(1)$, and remain very useful in their own right.

## 1. Stacks

Stacks are linear data structures which only permit access to one "end" of the data collection. We can only append ("push") items onto the tail end (a.k.a. the "top") of a stack, and only the most recently added item can be removed ("popped"). The last item to be pushed onto a stack is therefore the first one to be popped off, which is why we refer to stacks as last-in, first out (LIFO) structures.

In [6]:
# array-backed implementation

class Stack:
    def __init__(self):
        self.data = []
        
    def push(self, val):
        self.data.append(val)
    
    def pop(self):
        assert(len(self.data)>= 1)
        ret = self.data[-1]
        del self.data[-1]
        return ret
        # return self.data.pop()
    
    def empty(self):
        return len(self.data) == 0
    
    def __bool__(self):
        return not self.empty()

In [7]:
s = Stack()
for x in range(10):
    s.push(x)

In [8]:
while s:
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


In [15]:
# linked implementation

class Stack:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.top = None

    def push(self, val):
        self.top = Stack.Node(val, next= self.top)
        
    def pop(self):
        assert(self.top)
        ret = self.top.val
        self.top = self.top.next 
        return ret
    
    def empty(self):
        return self.top == None # empty is True
    
    def __bool__(self):
        return not self.empty() 
    # empty turns into False so we can exit the loop

In [16]:
s = Stack()
for x in range(10):
    s.push(x)

In [17]:
while s: # once it is empty then exit
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


### ... for delimiter pairing

e.g., `'(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))'`

In [ ]:
def check_parens(expr):
    s = Stack()
    for c in expr:
        if c == '(':
            s.push(c)
        elif c == ')':
            if not s or s.pop() != '(': # empty or not opening parentheses
                return False
        else:
            pass
    return s.empty()

In [ ]:
check_parens('()')

In [ ]:
check_parens('((()))')

In [ ]:
check_parens('()(()()(()))')

In [ ]:
check_parens('(')

In [ ]:
check_parens('())')

In [ ]:
check_parens('(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))')

### ... for postfix expression (aka "reverse polish notation") evaluation

e.g., `'(1 + 2) * 5'` $\rightarrow$ `'1 2 + 5 *'`

In [18]:
def eval_postfix(expr):
    s = Stack()
    toks = expr.split()
    for t in toks:
        if t == '*':
            s.push(s.pop()*s.pop())
        elif t == '+':
            s.push(s.pop()+s.pop())
        else:
            s.push(int(t))
    return s.pop() # the result is returned

In [20]:
eval_postfix('1 2 + 5 *')# (1+2)*5

15

In [19]:
eval_postfix('1 2 5 * +') # (5*2)+1

11

In [21]:
# ((1 + 2) * (3 + 2)) * 10
eval_postfix('1 2 + 3 2 + * 10 *')

150

### ... for tracking execution and backtracking

In [22]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

def parse_maze(maze_str):
    grid = []
    for line in maze_str.split('\n'):
        grid.append(['# IO'.index(c) for c in line.strip()])
    return grid

def print_maze(grid):
    for r in grid:
        print(''.join('# IO!+'[c] for c in r))

In [23]:
parse_maze(maze_str)

[[0, 0, 0, 0, 0, 0],
 [2, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 3],
 [0, 0, 0, 0, 0, 0]]

In [24]:
maze = parse_maze(maze_str)
maze[1][1] = 4
print_maze(maze)

######
I!   #
# ## #
# ####
#    O
######


In [25]:
class Move:
    def __init__(self, frm, to):
        self.frm = frm
        self.to  = to
        
    def __repr__(self):
        return '({},{}) -> ({},{})'.format(self.frm[0], self.frm[1],
                                           self.to[0],  self.to[1])

def moves(maze, place):
    moves = [Move(place, (place[0]+d[0], place[1]+d[1]))
            for d in ((-1, 0), (1, 0), (0, -1), (0, 1))
            if place[0]+d[0] in range(len(maze)) and 
               place[1]+d[1] in range(len(maze[0])) and
               maze[place[0]+d[0]][place[1]+d[1]] in (1, 2, 3)]
    return moves

In [30]:
from time import sleep
from IPython.display import clear_output

def visit(maze, loc):
    """Annotates a loc in the maze as visited"""
    maze[loc[0]][loc[1]] = 5
    
def mark(maze, loc):
    """Annotates a loc in the maze as being of interest"""
    if maze[loc[0]][loc[1]] != 3:
        maze[loc[0]][loc[1]] = 4
    
def display(maze):
    clear_output(True)
    print_maze(maze)
    sleep(1)

def solve_maze(maze, entry):
    for m in moves(maze, entry):
        save_move(m)
    visit(maze, entry)
    while not out_of_moves():
        move = next_move()
        if maze[move.to[0]][move.to[1]] == 3:
            return True
        display(maze)
        visit(maze, move.to)
        for m in moves(maze, move.to):
            mark(maze, m.to)
            save_move(m)
    return False

In [31]:
move_stack = Stack()

def save_move(move):
    move_stack.push(move)

def next_move():
    return move_stack.pop()

def out_of_moves():
    return move_stack.empty()

In [32]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

In [33]:
solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#+++!O
######


True

In [34]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

In [ ]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

## 2. Queues

Queues are linear data structures wherein we are only permitted to append ("enqueue") items onto the rear, and remove ("dequeue") items from the front. The oldest item still in a queue is therefore the next one to be dequeued, which is why we refer to a queue as a first-in, first-out (FIFO) structure. It is helpful to think of a queue as being the model for a line at a typical supermarket checkout aisle (first customer in, first customer to be checked out).

In [ ]:
# array-backed implementation

class Queue:
    def __init__(self):
        self.data = []

    def enqueue(self, val):
        # option 1: O(1)
        # self.data.append(val)
        
        # option 2: O(N)
        self.data.insert(0,val)
        
    def dequeue(self):
        assert(self.data)
        # option 1: O(N)
        # ret = self.data[0]
        # del = self.data[0]
        # return ret
        
        # option 2: O(1)
        ret = self.data[-1]
        del = self.data[-1]
        return ret
    
    def empty(self):
        return len(self.data) == 0
    
    def __bool__(self):
        return not self.empty()

In [ ]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [ ]:
while q:
    print(q.dequeue())

In [ ]:
# linked implementation

class Queue:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.head = self.tail = None

    def enqueue(self, val):
        n = Queue.Node(val)
        if not self.head: # if empty
            self.head = self.tail = n
        else:
            self.tail.next = self.tail = n
            
    def dequeue(self):
        assert(self.head)
        ret = self.head.val
        self.head = self.head.next
        if not self.head: # if the head is None, set the tail to None too
            self.tail = None
        return val
    
    def empty(self):
        return self.head == None
    
    def __bool__(self):
        return not self.empty()

In [ ]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [ ]:
while q:
    print(q.dequeue())

### ... for tracking execution and backtracking

In [ ]:
move_queue = Queue()

def save_move(move):
    move_queue.enqueque(move)

def next_move():
    return move_queue.dequeue()

def out_of_moves():
    return move_queue.empty()

In [ ]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

In [ ]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

In [ ]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

In [ ]:
solve_maze(parse_maze(maze_str), (1, 0))

### ... for doling out work

In [ ]:
from threading import Thread, Lock
from time import sleep
import random

lock = Lock()
def worker_fn(cid, q):
    while True:
        try:
            with lock:
                work = q.dequeue()
        except: # queue is empty
            sleep(1)
            continue
        if work == 'Stop':
            print('Consumer', cid, 'stopping.')
            break
        else:
            print('Consumer', cid, 'processing', work)
            sleep(random.random())

work_queue = Queue()
for i in range(5):
    Thread(target=worker_fn, args=(i, work_queue)).start()

In [ ]:
import threading
threading.active_count()

In [ ]:
for i in range(10):
    with lock:
        work_queue.enqueue(i)

In [ ]:
for i in range(5):
    with lock:
        work_queue.enqueue('Stop')

### ... for fair scheduling (aka "round-robin" scheduling)

In [ ]:
from random import randint
from time import sleep

task_queue = Queue()
for i in range(3):
    task_queue.enqueue(('Job {}'.format(i), randint(3, 6)))

In [ ]:
n = task_queue.head
while n:
    print(n.val)
    n = n.next

In [ ]:
while not task_queue.empty():
    job, time_left = task_queue.dequeue()
    print('Running', job)
    sleep(1)
    time_left -= 1
    if time_left > 0:
        print('Re-queueuing', job, 'with remaining time =', time_left)
        task_queue.enqueue((job, time_left))
    else:
        print('*', job, 'done')

## 3. Run-time analysis

Stack & Queue implementations:

- Insertion (push and enqueue) = $O(1)$
- Deletion (pop and dequeue) = $O(1)$

## 4. Closing remarks